<a href="https://colab.research.google.com/github/nomura-st/gas_questionnaire/blob/main/%E5%85%AC%E9%96%8B%E6%98%A0%E7%94%BB%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re

from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

def selectRecurse(baseurl, url, selector):
  html = requests.get(url)
  # 取得したHTMLをBeautifulSoupを使ってパースします。
  soup = BeautifulSoup(html.content, "html.parser")
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  elem = soup.select(selector)

  # iframeの中を検索
  for iframe in soup.select("iframe"):
    urlInFrame = iframe.attrs['src']
    if not re.search("^//", urlInFrame) and re.search("^/", urlInFrame):
      elementsInFrame = selectRecurse (baseurl, baseurl + urlInFrame, selector)
      elem = elem + elementsInFrame

  return elem

def select(url, selector):
  html = requests.get(url)
  # 取得したHTMLをBeautifulSoupを使ってパースします。
  soup = BeautifulSoup(html.content, "html.parser")
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  elem = soup.select(selector)

  return elem

In [7]:
# ターゲットWebページのURLを指定します。
baseurl = "https://eiga.com/theater/10"
path = "/100101/3229/"

# パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
elem = select(baseurl + path, "main .content-container section[data-title]")
obj = map(lambda e: {
    "name": e["data-title"],
    "types": list(map(lambda t: {
        "type": t.select(".movie-type")[0].get_text("/") if len(t.select(".movie-type")) > 0 else "",
        "date": list(map(lambda d: d["data-time"], t.select("td [data-time]"))),
    }, e.select(".movie-schedule"))),
  }, elem)
  
# 抽出した要素ひとつひとつから情報を取得します。
for o in obj:
  print (o)

{'name': 'アバター\u3000ウェイ・オブ・ウォーター', 'types': [{'type': '字幕', 'date': ['1672661100', '1672702200', '1672747500', '1672788600', '1672833900']}, {'type': '吹替', 'date': ['1672733700', '1672820100']}, {'type': '3D/字幕', 'date': ['1672724700', '1672811100']}, {'type': '4D/3D/吹替', 'date': ['1672659300', '1672705800', '1672720200', '1672745700', '1672792200', '1672806600', '1672832100']}]}
{'name': 'THE FIRST SLAM DUNK', 'types': [{'type': '', 'date': ['1672655700', '1672661100', '1672665300', '1672704000', '1672713900', '1672723800', '1672737300', '1672742100', '1672747500', '1672751700', '1672790400', '1672800300', '1672810200', '1672823700', '1672828500', '1672833900', '1672838100']}]}
{'name': 'Dr.コトー診療所', 'types': [{'type': '', 'date': ['1672662000', '1672704900', '1672716300', '1672726800', '1672748400', '1672791300', '1672802700', '1672813200', '1672834800']}]}
{'name': 'すずめの戸締まり', 'types': [{'type': '', 'date': ['1672662900', '1672702200', '1672712400', '1672722900', '1672732800', '16727